[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/emiletimothy/Caltech-CS155-2023/blob/main/set5/set5_prob3.ipynb)


## Set 5
## 3. Word2Vec **Principles**

#### Preparation

In [ ]:
# download the helper function
!wget -O P3CHelpers.py https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/P3CHelpers.py

--2023-02-27 07:41:38--  https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/P3CHelpers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4939 (4.8K) [text/plain]
Saving to: ‘P3CHelpers.py’

P3CHelpers.py       100%[===================>]   4.82K  --.-KB/s    in 0s      

2023-02-27 07:41:38 (38.4 MB/s) - ‘P3CHelpers.py’ saved [4939/4939]



In [ ]:
# download the dataset
!wget -O dr_seuss.txt https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/data/dr_seuss.txt

--2023-02-27 07:41:38--  https://raw.githubusercontent.com/emiletimothy/Caltech-CS155-2023/main/set5/data/dr_seuss.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8810 (8.6K) [text/plain]
Saving to: ‘dr_seuss.txt’

dr_seuss.txt        100%[===================>]   8.60K  --.-KB/s    in 0s      

2023-02-27 07:41:38 (84.5 MB/s) - ‘dr_seuss.txt’ saved [8810/8810]



In [ ]:
import numpy as np
from P3CHelpers import *
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

#### Problem D:
Fill in the generate_traindata and find_most_similar_pairs functions.

In [ ]:
def get_word_repr(word_to_index, word):
    """
    Returns one-hot-encoded feature representation of the specified word given
    a dictionary mapping words to their one-hot-encoded index.

    Arguments:
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        word:          Word whose feature representation we wish to compute.

    Returns:
        feature_representation:     Feature representation of the passed-in word.
    """
    unique_words = word_to_index.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[word_to_index[word]] = 1
    return feature_representation

def generate_traindata(word_list, word_to_index, window_size=4):
    """
    Generates training data for Skipgram model.

    Arguments:
        word_list:     Sequential list of words (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window. Defaults to 2
                       (use the default value when running your code).

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training
                              points (one-hot-encoded vectors) and their corresponding output_word
                              (also one-hot-encoded vectors)

    """
    trainX = []
    trainY = []

    ##############################################################
    # TODO: Implement this function, populating trainX and trainY
    ##############################################################

    for i in range(len(word_list)):
      for j in range(1,window_size+1):
        if i-j >= 0:
          y = get_word_repr(word_to_index, word_list[i-j])
          trainY.append(y)
          trainX.append(get_word_repr(word_to_index, word_list[i]))
        if i+j < len(word_list):
          y = get_word_repr(word_to_index, word_list[i+j])
          trainY.append(y)
          trainX.append(get_word_repr(word_to_index, word_list[i]))
    return np.array(trainX), np.array(trainY)

In [ ]:
def find_most_similar_pairs(filename, num_latent_factors):
    """
    Find the most similar pairs from the word embeddings computed from
    a body of text

    Arguments:
        filename:           Text file to read and train embeddings from
        num_latent_factors: The number of latent factors / the size of the embedding
    """
    # Load in a list of words from the specified file; remove non-alphanumeric characters
    # and make all chars lowercase.
    sample_text = load_word_list(filename)

    # Create word dictionary
    word_to_index = generate_onehot_dict(sample_text)
    print("Textfile contains %s unique words"%len(word_to_index))
    # Create training data
    trainX, trainY = generate_traindata(sample_text, word_to_index)

    ##############################################################
    # TODO: 1) Create and train model in Pytorch.
    ##############################################################

    # vocab_size = number of unique words in our text file. Will be useful
    # when adding layers to your neural network
    vocab_size = len(word_to_index)

    model = nn.Sequential(
      nn.Linear(vocab_size, num_latent_factors),
      nn.Linear(num_latent_factors, vocab_size),
    )

    model.train()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    train_dataset = TensorDataset(torch.Tensor(trainX),torch.Tensor(trainY))
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

    for epoch in range(10):
      for batch_idx, (data, target) in enumerate(train_loader):
        # Erase accumulated gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calculate loss
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()

        # Weight update
        optimizer.step()
    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))


    ##############################################################
    # TODO: 2) Extract weights for hidden layer
    ##############################################################

    # set weights variable below
    weights = model.state_dict()['1.weight']
    print(weights.size())

    #weight = model.state_dict()['0.weight']
    #print(weight.size())

    # Find and print most similar pairs
    similar_pairs = most_similar_pairs(weights, word_to_index)
    for pair in similar_pairs[:30]:
        print(pair)

### Problem E-H:
Run your model on drseuss.txt and answer questions from E through H.

In [ ]:
find_most_similar_pairs('dr_seuss.txt', 10)

Textfile contains 308 unique words
Train Epoch: 10  Loss: 3.0943
torch.Size([308, 10])
Pair(wink, pink), Similarity: 0.98405004
Pair(pink, wink), Similarity: 0.98405004
Pair(man, mr), Similarity: 0.98388916
Pair(mr, man), Similarity: 0.98388916
Pair(just, jump), Similarity: 0.9806946
Pair(jump, just), Similarity: 0.9806946
Pair(heads, grows), Similarity: 0.9793353
Pair(grows, heads), Similarity: 0.9793353
Pair(likes, drink), Similarity: 0.978965
Pair(drink, likes), Similarity: 0.978965
Pair(brush, comb), Similarity: 0.97793293
Pair(comb, brush), Similarity: 0.97793293
Pair(eight, nine), Similarity: 0.9778442
Pair(nine, eight), Similarity: 0.9778442
Pair(today, gone), Similarity: 0.97737235
Pair(gone, today), Similarity: 0.97737235
Pair(goat, boat), Similarity: 0.9765057
Pair(boat, goat), Similarity: 0.9765057
Pair(zeep, gone), Similarity: 0.9751761
Pair(hot, sun), Similarity: 0.9749667
Pair(sun, hot), Similarity: 0.9749667
Pair(sleep, light), Similarity: 0.97488743
Pair(light, sleep), 